In [1]:
import pandas as pd

In [36]:
april = pd.read_csv('./big_data_ignore/clean_april.csv', low_memory=False)
may = pd.read_csv('./big_data_ignore/clean_may.csv', low_memory=False)

In [115]:
overlap = [complaint for complaint in may_complaint if complaint in april_complaint]
may_overlap = may.set_index('complaint_number').loc[overlap].reset_index()
april_overlap = april.set_index('complaint_number').loc[overlap].reset_index()

In [116]:
april_overlap['full_name'] = april_overlap.accused_officer_fname.astype(str) + '_' + \
                             april_overlap.accused_officer_lname.astype(str)
    
may_overlap['full_name'] = may_overlap.accused_officer_fname.astype(str) + '_' + \
                           may_overlap.accused_officer_lname.astype(str)

In [117]:
cols = [col for col in april_overlap.columns if col != 'complaint_number']
group = april_overlap.groupby('complaint_number')
df_list = [group[col].unique().reset_index() for col in cols]

april_summary = df_list[0]
for col_df in df_list[1:]:
    april_summary = pd.merge(april_summary, col_df, on='complaint_number')
    
    
cols = [col for col in may_overlap.columns if col != 'complaint_number']
group = may_overlap.groupby('complaint_number')
df_list = [group[col].unique().reset_index() for col in cols]

may_summary = df_list[0]
for col_df in df_list[1:]:
    may_summary = pd.merge(may_summary, col_df, on='complaint_number')

In [118]:
def get_names(nlist, fname=True):
    if fname:
        return [name.split('_')[0] for name in nlist]
    else:
        return [name.split('_')[1] for name in nlist]
    
april_summary.accused_officer_fname = april_summary.full_name.apply(get_names)
april_summary.accused_officer_lname = april_summary.full_name.apply(get_names, fname=False)
may_summary.accused_officer_fname = may_summary.full_name.apply(get_names)
may_summary.accused_officer_lname = may_summary.full_name.apply(get_names, fname=False)

april_summary.drop('full_name', axis=1, inplace=True)
may_summary.drop('full_name', axis=1, inplace=True)

In [119]:
cols = april_summary.columns.tolist()[1:]
april_summary[cols] = april_summary[cols].applymap(lambda x: x[0] if len(x)==1 else x)

cols = may_summary.columns.tolist()[1:]
may_summary[cols] = may_summary[cols].applymap(lambda x: x[0] if len(x)==1 else x)

In [122]:
april_summary.to_csv('./big_data_ignore/april_overlap.csv')
may_summary.to_csv('./big_data_ignore/may_overlap.csv')

In [125]:
len([crid for crid in april_complaint if crid in may_complaint])

161